In [1]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import nltk
import pandas as pd

In [2]:
# load medium library english for spaCy
nlp = spacy.load("en_core_web_md")
# download Vader for nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\pmerk\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
path = "C:\\Users\\scheib\\Desktop\\challenges_scm\\daten\\C-SCM-DATA-Candidates_Evaluation_Anonymized_SS21.xlsx"

In [4]:
data = pd.read_excel(path, engine='openpyxl')
statements = data['Evaluation Statement'].tolist()

In [66]:
text = "He is the most creative person in the WHOLE world."
test = text + " " +  statements[0]
doc = nlp(test)

In [67]:
doc

He is the most creative person in the WHOLE world. Based on his international background 00580346 Flores has proven that he is an excellent contributor in various corporate and business roles and that he navigates smoothly in any diverse cultural context, embracing the value of multiple perspectives and experiences. For example, 00580346 has repeatedly adopted new roles, quickly contributing and taking over special tasks and challenges. Prior to joining corporate strategy team, 00580346 has ensured an active exchange with his predecessor with a keen interest to gather as many insights as possible. Due to this intense preparation, 00580346 covered the routine tasks very quickly and took up the lead role in the quarterly OD benchmark BoD paper with an impressing and recognized quality. 00580346’s openness to learn new topics and processes was reflected in the continued support of major corporate portfolio projects and he volunteered in activities beyond his role, e.g. BASF 4.0 activities

In [68]:
import itertools as it
import typing as tp

VERB_POS = {"VERB", "AUX"}
SPLIT_WORDS = {"and", "although", "but", "however", "except", "also", "nevertheless"}

language_model = "en_core_web_lg"
nlp = spacy.load(language_model)

ls_sentences = []

for sent in doc.sents:
        index = 0
        for t in sent.as_doc():
            if(t.text in SPLIT_WORDS or t.pos_ == "PUNCT"):
                ls_sentences.append(sent.as_doc()[index:t.i])
                index = t.i 

print(ls_sentences[0].text, cosine_similarity(ls_sentences[0].vector,nlp("creative").vector))

from scipy import spatial

cosine_similarity = lambda x, y: 1 - spatial.distance.cosine(x, y)

open_ = ['outgoing', 'curious', 'open-minded', 'broad-minded', 'honest', 'empathetic', 'respectful', 'positivity', 'emotional intelleligence', 'interest', 'interested', 'adapting' , 'informative', 'sharing', 'feedback', 'honesty', 'trust', 'valuing', 'diversity', 'perspective']
responsible = ['decisions', 'decision-maker', 'supportive', 'prepared', 'proactive', 'reliable', 'trustworthy', 'discipline', 'respectable', 'committed', 'integrity', 'pushing', 'assertive', 'obligated', 'judicious' , 'organized', 'managing', 'consistent']
creative = ['creative', 'constructive', 'resourceful', 'imaginative', 'ingenious', 'canny', 'inventive', 'full of ideas', 'clever', 'adventurous', 'innovative', 'originative', 'visionary', 'fanciful', 'forward thinker', 'pioneering', 'fertile', 'mastermind', 'genius', 'go-ahead', 'witty', 'eccentrically', 'inspiring', 'stimulating', 'encouraging', 'rich in ideas', 'inspirational']
entrepreneurial = ['entrepreneurial', 'enterprising', 'entrepreneurially', 'profit-oriented', 'for-profit', 'profit-seeking', 'need for achievement', "self-efficacy", 'innovativeness', 'stress tolerant', 'need for autonomy', 'proactive', 'disruptive', 'personality', 'venturesome', 'prepared to take risks', 'visionary', 'goal-oriented', 'purposeful', 'active', 'engaged', 'maker', 'doer', 'self-starter', 'calm', 'passionate', 'positive', 'convinced']

threshold = 0.725

sent_creative = []
set_creative = set()
for l in ls_sentences:
    for c in creative:
        if(cosine_similarity(l.vector,nlp(c).vector) >= threshold):
            sent_creative.append((l, c ,cosine_similarity(l.vector,nlp(c).vector)))
            set_creative.add(l)

print(set_creative)
              

He is the most creative person in the WHOLE world 0.5731542706489563
{and processes was reflected in the continued support of major corporate portfolio projects, , embracing the value of multiple perspectives, Based on his international background 00580346 Flores has proven that he is an excellent contributor in various corporate, 00580346’s openness to learn new topics, , he has proactively developed a sound business plan for the part of his portfolio associated with the upcoming BASF Colors & Pigment carveout, He is the most creative person in the WHOLE world, , 00580346 has already proven in his first year in the current position that he can handle a far more significant business portfolio, and that he navigates smoothly in any diverse cultural context, , 00580346 has ensured an active exchange with his predecessor with a keen interest to gather as many insights as possible, and took up the lead role in the quarterly OD benchmark BoD paper with an impressing, and this should provide

In [53]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def sentiment_scores(sentence):
  
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
  
    # polarity_scores method of SentimentIntensityAnalyzer
    # oject gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)
      
    print("Overall sentiment dictionary is : ", sentiment_dict)
    print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative")
    print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral")
    print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive")
  
    print("Sentence Overall Rated As", end = " ")
  
    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.05 :
        print("Positive")
  
    elif sentiment_dict['compound'] <= - 0.05 :
        print("Negative")
  
    else :
        print("Neutral")   

In [56]:
for s in set_creative:
    print(s.text)
    sentiment_scores(s.text)

Based on his international background 00580346 Flores has proven that he is an excellent contributor in various corporate
Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.821, 'pos': 0.179, 'compound': 0.5719}
sentence was rated as  0.0 % Negative
sentence was rated as  82.1 % Neutral
sentence was rated as  17.9 % Positive
Sentence Overall Rated As Positive
and took up the lead role in the quarterly OD benchmark BoD paper with an impressing
Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.811, 'pos': 0.189, 'compound': 0.5423}
sentence was rated as  0.0 % Negative
sentence was rated as  81.10000000000001 % Neutral
sentence was rated as  18.9 % Positive
Sentence Overall Rated As Positive
00580346’s openness to learn new topics
Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.676, 'pos': 0.324, 'compound': 0.34}
sentence was rated as  0.0 % Negative
sentence was rated as  67.60000000000001 % Neutral
sentence was rated as  32.4 % Positive
Sentence Overall Rated As 

In [ ]:
positive sentiment : (compound score >= 0.05)
neutral sentiment : (compound score > -0.05) and (compound score < 0.05)
negative sentiment : (compound score <= -0.05)